In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import IPython.display as ipd
import librosa
import librosa.display

In [ ]:
dataset_path = r"E:\CHANDRU\chand proj\MOUD\audioFiles"
target_data = pd.read_csv("E:\\CHANDRU\\chand proj\\MOUD\\target_audio.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Fit and transform the data
#colors_encoded = le.fit_transform(target)
target_data['class'] = le.fit_transform(target_data['class'])
target_data['class'].head()

In [ ]:
def features_extractor(file):
    audio, sr = librosa.load(filepath)
#     audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

    mfccs_features = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
 
    return mfccs_scaled_features


In [ ]:
def plotFun(file):
    audio, sr = librosa.load(filepath)
    S = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=128, fmax=8000)
    S_dB_mel = librosa.amplitude_to_db(S, ref=np.max)
 
    fig,ax = plt.subplots(figsize=(10, 5))
    img = librosa.display.specshow(S_dB_mel, y_axis='log', x_axis='time', ax=ax)
    ax.set_title('Power spectrogram')
    fig.colorbar(img, ax=ax, format='%+2.0f dB')
    plt.show()

In [ ]:

a=target_data["instance"].to_numpy()
b=[]
for filename in os.listdir(dataset_path):
    if filename.endswith(".wav"):
        filepath = os.path.join(dataset_path, filename)
        b.append(filename)
ar1=[]
for i in b:
    st1=i.split(".")[0]
    ar1.append(st1)
ar2=np.array(ar1)

d= np.intersect1d(a,ar2)
z=np.setxor1d(a,ar2)

In [ ]:
import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
import numpy as np
cnt=0
#extracted_features=[]
for filename in os.listdir(dataset_path):
    if filename.endswith(".wav"):
        str1=filename.split(".")[0]
        if str1 in z:
            print(str1)
        else:
            cnt=cnt+1
            print(cnt)        
        # Load the audio file and resample it to the target sampling rate
            filepath = os.path.join(dataset_path, filename)
            plotFun(filepath)
            data=features_extractor(filepath)
        
            extracted_features.append([data])


In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature'])


In [ ]:
extracted_features_df.head()

In [ ]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(target_data[['class']])

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)


In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from keras.utils import to_categorical


In [ ]:
model=Sequential()
### firstlayer
#model.add(Dense(100,input_shape=(X_train.shape[1],)))
model.add(Dense(100,input_shape=(40,)))

model.add(Activation("relu"))
model.add(Dropout(0.5))
### second layer
model.add(Dense(200))
model.add(Activation("relu"))
model.add(Dropout(0.5))
### third layer 
model.add(Dense(100))
model.add(Activation("relu"))

model.add(Dropout(0.5))
###final layer
model.add(Dense(2))
model.add(Activation("softmax"))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [ ]:
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy",metrics=["accuracy"],optimizer="adam")

In [ ]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs=80
num_batch_size=32

checkpoint_callback = ModelCheckpoint(filepath="best_model1.h5",verbose=1, monitor='val_accuracy', save_best_only=True)

start=datetime.now()
model.fit(X_train,y_train,batch_size=num_batch_size,epochs=num_epochs,validation_data=(X_train,y_train),callbacks=[checkpoint_callback] )
duration=datetime.now() - start
print("training completed in time:",duration)

In [ ]:
from sklearn.preprocessing import LabelEncoder
filename=r"E:\CHANDRU\chand proj\MOUD\audioFiles\134_makeup_3.wav"
audio, sr = librosa.load(filepath) 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features.shape)
predicted_label=model.predict(mfccs_scaled_features)
predicted_label = np.argmax(predicted_label)
prediction_class = le.inverse_transform([predicted_label])
print(prediction_class)


In [ ]:

# loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
# print('Test accuracy:', accuracy)